In [1]:
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np

In [492]:
with open(r'api_key.txt', 'r') as file:
    api_key = file.readline()


In [123]:
url_austin = r'https://en.wikipedia.org/wiki/List_of_Austin_neighborhoods'

In [186]:
table_austin = pd.read_html(url_austin)
table_austin = table_austin[0]
table_austin.drop(['COA ID#[nb 1]'], axis=1, inplace = True)
austin_neighborhoods = table_austin
austin_neighborhoods.rename(columns={'Name':'Neighborhood'}, inplace=True)
austin_neighborhoods.head()

,Neighborhood
0,Bryker Woods
1,Caswell Heights
2,Downtown Austin
3,Eastwoods
4,Hancock


In [17]:
url_dallas = r'https://en.wikipedia.org/wiki/List_of_neighborhoods_in_Dallas'

In [127]:
table_dallas = pd.read_html(url_dallas)
table_dallas_concat = pd.DataFrame(columns=["Neighborhood"])
for i in range(1, len(table_dallas)):
    temp_table = pd.DataFrame(table_dallas[i].iloc[:,1])
    temp_table.rename(columns={temp_table.columns[0]:"Neighborhood"}, inplace=True)
    table_dallas_concat = table_dallas_concat.append(temp_table, ignore_index=True)
    temp_table = []
dallas_neighborhoods = table_dallas_concat
dallas_neighborhoods.head()

,Neighborhood
0,Baylor District
1,The Cedars
2,Civic Center District
3,Dallas Arts District
4,Dallas Farmers Market


In [138]:
url_chicago = r'https://en.wikipedia.org/wiki/List_of_neighborhoods_in_Chicago'
#url_chicago = r'https://en.wikipedia.org/w/index.php?title=List_of_neighborhoods_in_Chicago&action=edit&section=1'

In [147]:
table_chicago = pd.read_html(url_chicago)
table_chicago = table_chicago[0]
table_chicago.drop(['Community area'], axis=1, inplace=True)
chicago_neighborhoods = table_chicago
chicago_neighborhoods.head()
#table_chicago.drop(['COA ID#[nb 1]'], axis=1, inplace=True)
#table_chicago.head(50)

,Neighborhood
0,Albany Park
1,Altgeld Gardens
2,Andersonville
3,Archer Heights
4,Armour Square


In [208]:

import re
import requests
from bs4 import BeautifulSoup

In [514]:
jacksonville_neighborhoods = pd.DataFrame(columns=['Neighborhood'])
jacksonville_neighborhoods

,Neighborhood


In [515]:
url_string=r'https://www.conservapedia.com/Neighborhoods_of_Jacksonville,_Florida'
page=requests.get(url_string)
page=page.text
soup=BeautifulSoup(page,features="html.parser")
results=soup.find_all("li", class_="toclevel-2")
for index, item in enumerate(results):
    area = item.find("span", class_="toctext").text
    jacksonville_neighborhoods = jacksonville_neighborhoods.append({'Neighborhood':area}, ignore_index=True) 
jacksonville_neighborhoods.head()
#for item in results:
#    print(item.text)

,Neighborhood
0,Argyle
1,Avondale
2,Cedar Hills
3,Confederate Point
4,Lake Shore


In [516]:
jacksonville_neighborhoods['Neighborhood'] = jacksonville_neighborhoods['Neighborhood'].map(lambda x: x +', Jacksonville, '+ 'Fl')
#mod_jacksonville_neighborhoods.rename(columns={0:'Neighborhood'}, inplace=True)
jacksonville_neighborhoods.head()

,Neighborhood
0,"Argyle, Jacksonville, Fl"
1,"Avondale, Jacksonville, Fl"
2,"Cedar Hills, Jacksonville, Fl"
3,"Confederate Point, Jacksonville, Fl"
4,"Lake Shore, Jacksonville, Fl"


In [257]:
import json
from pandas.io.json import json_normalize

In [522]:
def get_lng_lat_zip(neighborhoods):
    location_lng_lat=pd.DataFrame(columns=['Latitude', 'Longitude'])
    for i in range(len(neighborhoods)):
        url = r'https://maps.googleapis.com/maps/api/geocode/json?address={}&key={}'.format(neighborhoods['Neighborhood'][i], api_key)
        r = requests.get(url)
        json_r = json.loads(r.text)
        try:
            location_pd = json_normalize(json_r['results'])
            location_lng_lat_temp =  location_pd[['geometry.location.lat', 'geometry.location.lng']]
            location_lng_lat_temp.columns=['Latitude', 'Longitude']
            location_lng_lat = pd.concat([location_lng_lat, location_lng_lat_temp], ignore_index=True)
        except:
            continue
            
    return location_lng_lat                              

In [523]:
jacksonville_neighborhoods_zip = get_lng_lat_zip(jacksonville_neighborhoods)
jacksonville_neighborhoods_zip.head()

,Latitude,Longitude
0,30.196921,-81.756282
1,30.296769,-81.710315
2,30.253614,-81.756138
3,30.256893,-81.736310
4,30.280630,-81.726734


In [542]:
jacksonville_nb_lng_lat = pd.concat([jacksonville_neighborhoods,
                                     jacksonville_neighborhoods_zip],
                                    axis=1)
jacksonville_nb_lng_lat.head()

,Neighborhood,Latitude,Longitude
0,"Argyle, Jacksonville, Fl",30.196921,-81.756282
1,"Avondale, Jacksonville, Fl",30.296769,-81.710315
2,"Cedar Hills, Jacksonville, Fl",30.253614,-81.756138
3,"Confederate Point, Jacksonville, Fl",30.256893,-81.736310
4,"Lake Shore, Jacksonville, Fl",30.280630,-81.726734


In [538]:
def zip_nb(neighborhoods):
    #zip_pd = pd.DataFrame(columns=['zip_code'])
    zip_pd_temp = []
    for i in range(len(neighborhoods)):
        url_2 = 'https://maps.googleapis.com/maps/api/geocode/json?latlng={},{}&key={}'.format(neighborhoods['Latitude'][i], neighborhoods['Longitude'][i],api_key)
        r = requests.get(url_2)
        json_r = json.loads(r.text)
        location_pd_3 = json_normalize(json_r['results'][0]['address_components'])
        count=0
        for line in location_pd_3.values:
            if 'postal_code' in line[2]:
                zip_code = line[0]
                count=1
            if count != 1:
                zip_code = 'NA'    
        zip_pd_temp.append(zip_code)
        count=0
    zip_pd = pd.DataFrame(zip_pd_temp, columns = ['zip_code'])
    return(zip_pd)  

In [539]:
jacksonville_zip_pd = zip_nb(jacksonville_nb_lng_lat)
jacksonville_zip_pd.head()

,zip_code
0,32244
1,32205
2,32210
3,32210
4,32210


In [543]:
jacksonville_zip_lng_lat = pd.concat([jacksonville_nb_lng_lat,
                                     jacksonville_zip_pd],
                                    axis=1)
jacksonville_zip_lng_lat.head()

,Neighborhood,Latitude,Longitude,zip_code
0,"Argyle, Jacksonville, Fl",30.196921,-81.756282,32244
1,"Avondale, Jacksonville, Fl",30.296769,-81.710315,32205
2,"Cedar Hills, Jacksonville, Fl",30.253614,-81.756138,32210
3,"Confederate Point, Jacksonville, Fl",30.256893,-81.736310,32210
4,"Lake Shore, Jacksonville, Fl",30.280630,-81.726734,32210


In [482]:
dallas_neighborhoods_zip = get_lng_lat_zip(dallas_neighborhoods)
dallas_neighborhoods_zip.head()

C:\Users\gtm12\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Longitude,Latitude,zip_code
0,33.537393,-99.278558,NA
1,38.379907,-97.652883,67460
2,37.781553,-122.415643,94102
3,32.789731,-96.798262,NA
4,32.777505,-96.789975,75201


In [484]:
dallas_nb_lng_lat = pd.concat([dallas_neighborhoods,
                                     dallas_neighborhoods_zip],
                                    axis=1)
dallas_nb_lng_lat

,Neighborhood,Longitude,Latitude,zip_code
0,Baylor District,33.537393,-99.278558,NA
1,The Cedars,38.379907,-97.652883,67460
2,Civic Center District,37.781553,-122.415643,94102
3,Dallas Arts District,32.789731,-96.798262,NA
4,Dallas Farmers Market,32.777505,-96.789975,75201
5,Deep Ellum,32.784392,-96.780185,NA
6,Design District,25.813422,-80.193428,NA
7,Main Street District,32.781379,-96.796922,NA
8,Reunion District,32.776934,-96.806842,75202
9,Riverfront District,40.484183,-86.130962,46901
